In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Detailed plots

In [2]:
log_dir = "../features/"

models = ['MAE', 'MoCo-V3', 'MSN', 'DINO']
modes = ['linear', 'finetuned']

results = {}

for model in models:
    results[model] = {}
    for mode in modes:
        file = log_dir + model + "/" + mode + "/knn_results.csv"
        results[model][mode] = pd.read_csv(file, index_col=0)

In [4]:
import textwrap

In [5]:
fig_dir = '../figures/ICML/'

In [7]:
fig, axn = plt.subplots(1, 2, figsize=(28, 11), dpi=500, sharex=True, sharey=True)
#cbar_ax = fig.add_axes([.9, .3, .01, .5])

#fig.supxlabel('common_x')
fig.supylabel('ImageNet Top-1 Accuracy', fontweight="bold", fontsize = 40, x=0.07)

sns.set(font_scale=1.5)
training = 'linear'

indexes = ['CLS', 'GAP w/o CLS']

for i, ax in enumerate(axn.flat):
    ax.set_xticks(range(0, 13, 1))
    ax.set_xticklabels([str(x) for x in range(1, 14, 1)])

    agg_mode = 'CLS_feats' if not i else 'GAP_WO_CLS_feats'

    for model in ['MAE', 'MoCo-V3', 'DINO', 'MSN']:
        f = sns.lineplot(x="ViT_block", y='top-1', data=results[model][training].loc[(results[model][training]['agg_mode'] == agg_mode) | (results[model][training]['agg_mode'] == 'linear')], marker='o', linewidth=5, ax=ax, label=model)


    ax.tick_params(axis='x', which='major', labelsize=30)
    ax.tick_params(axis='y', which='major', labelsize=30)

    ax.set_ylabel('')

    labels = [item.get_text() for item in ax.get_xticklabels()]
    labels[-1] = textwrap.fill('Probe', 5)
    ax.set_xticklabels(labels, size=30)#, weight='bold')

    ax.set_xlabel('Layer', size=40)

    #ax.set_yticklabels(ax.get_yticks(), weight='bold')
    
    
    if(i<1):
        ax.get_legend().remove()
    else:
        leg = ax.legend(fontsize=30)
        leg.set_title('Pre-Training',prop={'size':30, 'weight':'bold'})

    # Create offset transform by 5 points in x direction
    dx = 20/72.; dy = 0/72. 
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)

    # apply offset transform to all x ticklabels.
    for tick_index, label in enumerate(ax.xaxis.get_majorticklabels()):
        if tick_index==12:
            label.set_transform(label.get_transform() + offset)

    letter = {0:'(A) ', 1:'(B) '}
    ax.set_title(letter[i] +indexes[i]+" features", fontdict={'fontweight':"bold", 'fontsize':40}, y=-0.24)

plt.show()

fig.savefig(fig_dir + 'arxiv_KNN_plot.pdf', dpi=500, bbox_inches = "tight")